# **Assignment 2 – Information Visualisation**
## Data Processing for CSO High Value Dataset (EAA17 Greenhouse Gas Emissions)

## Goal
The goal is to perform data manipulation outside of the Vega-Lite environment, using Python. Making clear what transformations / filtering I perform and making the processed data set available.

## Dictionary
"STATISTIC": Internal CSO identifier for the type of emission being measured.

"Statistic Label": Human-readable name of the emission type being measured.

"TLIST(A1)": Time period identifier, stores the year as a string in YYYY format (matches Year).

"Year": The calendar year of the measurement.

"C04273V05050": NACE sector classification code.

"NACE Rev. 2 Sector": Full name of the economic sector following NACE Rev. 2 classification. Includes sector code in parentheses.

"UNIT": Unit of measurement for the VALUE field.

"VALUE": Actual emissions measurement, the quantity of the specified emission type from the specified sector in the specified year, measured in thousand tonnes (or thousand tonnes of CO2 equivalent for greenhouse gases).

# **1. Import libraries**

I import the necessary packages and set up everything I need.

In [16]:
import pandas as pd
import numpy as np

# Display settings for clarity
pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", "{:,.2f}".format)

I then load the original downloaded CSO dataset saved as `eaa17_original.csv`, containing 1 record per property. Showing the number of rows and columns in dataset, and the first 5 rows for preview

In [17]:
# Load original downloaded CSO dataset
df_raw = pd.read_csv("eaa17_original.csv")

# Inspect structure
print("Shape:", df_raw.shape)
df_raw.head(5)

Shape: (10650, 8)


,STATISTIC,Statistic Label,TLIST(A1),Year,C04273V05050,NACE Rev. 2 Sector,UNIT,VALUE
0,EAA17C01,Carbon dioxide (CO2),2009,2009,01,"Crop and animal production, hunting and relate...",Thousand Tonnes,"1,153.40"
1,EAA17C01,Carbon dioxide (CO2),2009,2009,02,Forestry and logging (02),Thousand Tonnes,53.67
2,EAA17C01,Carbon dioxide (CO2),2009,2009,03,Fishing and aquaculture (03),Thousand Tonnes,119.67
3,EAA17C01,Carbon dioxide (CO2),2009,2009,0509,Mining and quarrying (05-09),Thousand Tonnes,58.15
4,EAA17C01,Carbon dioxide (CO2),2009,2009,1012,"Manufacture of food products, beverages and to...",Thousand Tonnes,929.04


# **2. Overview and Checks**
I need to display column names, data types, and number of non-missing values for each column. To better understand which columns have missing data, which are numeric or categorical, if data types are correct, and if there’s missing or inconsistent data.

In [18]:
df_raw.info()

print('\n Analysis numerical categories\n')
print(df_raw[['TLIST(A1)','Year','VALUE']].describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10650 entries, 0 to 10649
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   STATISTIC           10650 non-null  object 
 1   Statistic Label     10650 non-null  object 
 2   TLIST(A1)           10650 non-null  int64  
 3   Year                10650 non-null  int64  
 4   C04273V05050        10650 non-null  object 
 5   NACE Rev. 2 Sector  10650 non-null  object 
 6   UNIT                10650 non-null  object 
 7   VALUE               10650 non-null  float64
dtypes: float64(1), int64(2), object(5)
memory usage: 665.8+ KB

 Analysis numerical categories

       TLIST(A1)      Year      VALUE
count  10,650.00 10,650.00  10,650.00
mean    2,016.00  2,016.00   3,011.25
std         4.32      4.32  34,864.76
min     2,009.00  2,009.00       0.00
25%     2,012.00  2,012.00       0.01
50%     2,016.00  2,016.00       0.78
75%     2,020.00  2,020.00      22.28
m

The dataset contains greenhouse gas emissions broken down by year, sector, and gas type. For the purposes of this visualisation, emissions are analysed in kilotonnes of CO₂ equivalent.

# **3. Data Pre-Processing**

## 3.1 Handle Missing and Invalid Values
First of all, I replace placeholder text for missing data ('???', 'na', ...) with actual NaN values that pandas can recognise as missing. 

After that, I can count missing values in each column. Sort them from most to least missing to see which categories might be more problematic.

In [19]:
df_raw.replace(['???', 'N/A', 'na', 'NaN', 'nan', 'None', ''], np.nan, inplace=True)

missing_list = df_raw.isna().sum().sort_values(ascending=False)
print('Missing values by category:\n', missing_list)

Missing values by category:
 STATISTIC             0
Statistic Label       0
TLIST(A1)             0
Year                  0
C04273V05050          0
NACE Rev. 2 Sector    0
UNIT                  0
VALUE                 0
dtype: int64


## 3.2 Standardise Data Types
Then I drop the column `TLIST(A1)` as I already have all the information I need separated in `Year`. And the columns `STATISTIC` and `C04273V05050` as I already have all the information I need separated in `Statistic Label` and `NACE Rev. 2 Sector` respectively.

And I drop duplicates from the complete dataset (which could be scraped twice or repeated).

In [20]:
# Drop columns
df_raw.drop(columns=['TLIST(A1)'], inplace=True)
df_raw.drop(columns=['STATISTIC'], inplace=True)
df_raw.drop(columns=['C04273V05050'], inplace=True)

# Drop duplicates (just in case)
df_raw.drop_duplicates(inplace=True)
# Print data types to confirm correct formats
print(df_raw.dtypes)
print('\nRows after dropping duplicates:', len(df_raw))

Statistic Label        object
Year                    int64
NACE Rev. 2 Sector     object
UNIT                   object
VALUE                 float64
dtype: object

Rows after dropping duplicates: 10650


**Notes:**

- No duplicates were present in the dataset, as the number of rows stayed the same as before dropping them.

In [21]:
# Get unique values in the UNIT column
unique_units = df_raw['UNIT'].unique()

print("Unique values in UNIT column:")
print(unique_units)
print("\nValue counts:")
print(df_raw['UNIT'].value_counts())

Unique values in UNIT column:
['Thousand Tonnes' 'Tonnes']

Value counts:
UNIT
Thousand Tonnes    8520
Tonnes             2130
Name: count, dtype: int64


In [22]:
# Convert Tonnes to Thousand Tonnes
# Divide VALUE by 1000 for rows where UNIT is 'Tonnes'
df_raw.loc[df_raw['UNIT'] == 'Tonnes', 'VALUE'] = df_raw.loc[df_raw['UNIT'] == 'Tonnes', 'VALUE'] / 1000

# Update the UNIT column to 'Thousand Tonnes'
df_raw.loc[df_raw['UNIT'] == 'Tonnes', 'UNIT'] = 'Thousand Tonnes'

# Display final state
print("After conversion:")
print(df_raw['UNIT'].value_counts())

After conversion:
UNIT
Thousand Tonnes    10650
Name: count, dtype: int64


In [23]:
# Rename columns for clarity and easier Vega-Lite encoding
df = df_raw.rename(columns={
    "NACE Rev. 2 Sector": "NACE Sector",
    "VALUE": "Value",
    "UNIT": "Unit"
})

df.sample(10)

,Statistic Label,Year,NACE Sector,Unit,Value
2948,Hydrofluorocarbons in CO2 equivalent (HFC_CO2E),2020,Other Household activities,Thousand Tonnes,223.78
5980,Nitrous oxide (N2O),2018,"Manufacture of computer, electronic and optica...",Thousand Tonnes,0.00
1251,Carbon dioxide from biomass (CO2_BIO),2011,"Financial service activities, except insurance...",Thousand Tonnes,0.30
3179,Hydrofluorocarbons in CO2 equivalent (HFC_CO2E),2023,Employment activities (78),Thousand Tonnes,0.06
2950,Hydrofluorocarbons in CO2 equivalent (HFC_CO2E),2020,Accommodation and food service activities (55-56),Thousand Tonnes,3.85
9014,Sulphur hexafluoride and nitrogen trifluoride ...,2015,Total NACE activities,Thousand Tonnes,61.56
8155,Perfluorocarbons in CO2 equivalent (PFC_CO2E),2018,Residential care activities and social work ac...,Thousand Tonnes,0.00
6086,Nitrous oxide (N2O),2019,Scientific research and development (72),Thousand Tonnes,0.00
4918,Methane in CO2 equivalent (CH4_CO2E),2018,"Manufacture of motor vehicles, trailers and se...",Thousand Tonnes,0.04
1024,Carbon dioxide (CO2),2023,Land transport and transport via pipelines (49),Thousand Tonnes,"2,170.54"


In [24]:
# Ensure emissions are numeric
df["Value"] = pd.to_numeric(df["Value"], errors="coerce")

df = df.dropna(subset=["Value"])

print("After cleaning missing values:", df.shape)

After cleaning missing values: (10650, 5)


## 3.3 Check unique values

I need to know the unique values for teh Statistic Label and the NACE Sector, so that I can later chooose the correct type of visualisation for my dashboard.

In [25]:
unique_statistics = df["Statistic Label"].unique()

print("Unique values in 'Statistic Label':")
for value in sorted(unique_statistics):
    print("-", value)

print(f"\nTotal unique Statistic Labels: {len(unique_statistics)}")

Unique values in 'Statistic Label':
- Carbon dioxide (CO2)
- Carbon dioxide from biomass (CO2_BIO)
- Hydrofluorocarbons in CO2 equivalent (HFC_CO2E)
- Methane (CH4)
- Methane in CO2 equivalent (CH4_CO2E)
- Nitrous oxide (N2O)
- Nitrous oxide in CO2 equivalent (N2O_CO2E)
- Perfluorocarbons in CO2 equivalent (PFC_CO2E)
- Sulphur hexafluoride and nitrogen trifluoride in CO2 equivalent (NF3_SF6_CO2E)
- Total greenhouse gases in CO2 equivalent (GHG)

Total unique Statistic Labels: 10


In [26]:
unique_sectors = df["NACE Sector"].unique()

print("Unique values in 'NACE Sector':")
for value in sorted(unique_sectors):
    print("-", value)

print(f"\nTotal unique NACE Sectors: {len(unique_sectors)}")

Unique values in 'NACE Sector':
- Accommodation and food service activities (55-56)
- Activities auxiliary to financial services and insurance activities (66)
- Activities of extraterritorial organisations and bodies (99)
- Activities of households as employers; undifferentiated production by households (97-98)
- Activities of membership organisations (94)
- Advertising and market research (73)
- Air transport (51)
- Architectural and engineering activities; technical testing and analysis (71)
- Arts and entertainment activities; libraries, archives, museums; gambling activities (90-92)
- Audiovisual programme production; programming and broadcasting activities (59-60)
- Computer programming, consultancy, and information service activities (62-63)
- Construction (41-43)
- Crop and animal production, hunting and related service activities (01)
- Education (85)
- Electricity, gas, steam and air conditioning supply (35)
- Employment activities (78)
- Financial service activities, except i

## 3.4 Summary Statistics
Now I want to know the statistical summaries (count, mean, std, min, 25%, 50%, 75%, max, median) for numerical categories.

In [27]:
print("Final processed dataset shape:", df.shape)
df.describe()

Final processed dataset shape: (10650, 5)


,Year,Value
count,"10,650.00","10,650.00"
mean,"2,016.00",621.82
std,4.32,"4,558.82"
min,"2,009.00",0.00
25%,"2,012.00",0.00
50%,"2,016.00",0.11
75%,"2,020.00",12.09
max,"2,023.00","80,829.59"


# **4. Save the CSV**
Finally, I write the final cleaned data to a new CSV file.

In [28]:
# Export processed dataset for Vega-Lite dashboard
df.to_csv("eaa17_processed.csv", index=False)

print("Processed dataset saved as eaa17_processed.csv")

Processed dataset saved as eaa17_processed.csv
